In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numba
import seaborn as sns
from IPython.display import display, HTML
import helperfunctions # this is my own python file
from stdev_emsd import stdev_emsd # this is my own python file



display(HTML("<style>.container { width: 90% !important; }</style>")) # Set the custom CSS to make the notebook width 90% of the window size
sns.set_context('notebook')

In [13]:
filename_list = [r"C:\Users\pguest\Desktop\pickles\Data Taken 2024-07-22, 2024-07-22_ND4_2.avicontrol_tracer_nodrift.pkl",
                          r"C:\Users\pguest\Desktop\pickles\Data Taken 2024-07-22, 2024-07-22_ND4_1.avicontrol_tracer_nodrift.pkl"]

fps_list = [1,1]
scaling_list = [1,1]

fps = fps_list[0]
scaling = scaling_list[0]

In [14]:
df_list = [] # initialize list of dataframes
for filepath in filename_list:
    df_list.append(pd.read_pickle(filepath)) # read trajectory files

In [15]:
df_list[1]

,x,y,relative_to_frame,y_raw,x_raw,mass,size,ecc,signal,raw_mass,ep,frame,particle
frame,,,,,,,,,,,,,
1,863.462583,614.143976,0,614.421222,863.487138,322.869613,3.540442,0.096327,6.955712,3472.333333,0.050034,1,11
1,342.515677,643.246239,0,643.234734,342.524182,425.025144,3.104475,0.060856,12.250358,3782.000000,0.036484,1,10
1,1189.521955,665.200947,0,665.658910,1189.518161,182.924842,3.053068,0.131701,4.983197,3239.333333,0.069438,1,21
1,427.111685,729.644348,0,729.679719,427.072289,103.401330,3.359785,0.198292,2.595415,3084.333333,0.093581,1,9
1,885.457409,752.043353,0,752.332769,885.405527,184.066825,3.934245,0.161175,3.633581,3165.333333,0.079192,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,225.266188,22.109206,0,30.357527,228.311262,656.319175,3.841862,0.071511,9.564357,4755.000000,0.019809,1555,64
1555,239.254669,826.353795,0,834.661210,238.342275,879.270836,3.094236,0.028329,22.285880,4818.333333,0.019235,1555,4
1555,733.949855,994.508104,0,1005.247640,732.204067,255.730477,3.418307,0.131611,5.757186,3285.000000,0.064424,1555,143


In [17]:
stdev_emsd(df_list[1], mpp=scaling, fps = fps, detail = True)

,<x>,<y>,<x^2>,<y^2>,msd,N,lagt,std_<x>,std_<y>,std_<x^2>,std_<y^2>,std_msd
frame,,,,,,,,,,,,
1,0.000812,-0.000864,0.514937,0.574399,1.073078,38969.071443,1.0,0.075249,0.055495,0.259716,0.315502,0.516740
2,-0.000244,-0.005070,1.042632,1.092091,2.134723,25961.197431,2.0,0.099546,0.080687,0.326232,0.361018,0.632032
3,-0.001954,-0.008651,1.615127,1.662009,3.228223,18432.521824,3.0,0.104972,0.097090,0.315147,0.395829,0.613926
4,-0.002187,-0.007459,2.200167,2.202861,4.337312,14139.480837,4.0,0.133418,0.118203,0.364185,0.470033,0.701227
5,-0.006497,-0.003692,2.788697,2.754231,5.460197,11427.541618,5.0,0.170287,0.132530,0.491722,0.511386,0.790410
...,...,...,...,...,...,...,...,...,...,...,...,...
96,-0.232815,-0.142850,59.242479,55.718131,105.979312,565.626752,96.0,2.552077,1.971628,20.926556,19.138030,27.306254
97,-0.222601,-0.153594,58.058298,54.471834,107.255282,559.391496,97.0,2.609242,1.997759,21.132187,18.763457,27.356757
98,-0.247984,-0.155565,61.471045,57.861108,108.144764,553.283932,98.0,2.594520,1.991936,22.157254,19.728894,28.156108
